#### Libri2MixDatasat

##### imports

In [2]:
import glob
import random
import os
from tqdm import tqdm
import pandas as pd

##### load & unzip libri2mix

In [3]:
# ! cp /content/libri2mix_test_clean_8k_max_mix_clean.zip /content/drive/MyDrive/Classroom/

In [4]:
!unzip //content/drive/MyDrive/Classroom/libri2mix_test_clean_8k_max_mix_clean.zip

Archive:  //content/drive/MyDrive/Classroom/libri2mix_test_clean_8k_max_mix_clean.zip
   creating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/4970-29095-0033_2094-142345-0058.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/3575-170457-0055_3570-5695-0008.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/7729-102255-0017_4970-29093-0009.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/121-121726-0008_5105-28240-0008.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/237-126133-0025_1284-1180-0023.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/1995-1837-0010_5639-40744-0033.wav  
  inflating: content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/5683-32866-0027_2830-3980-0030.wav  
  inflating: content/LibriMix/st

In [5]:
!cp -r /content/content/LibriMix/storage_dir/Libri2Mix/wav8k/max/test/mix_clean/ /content/

In [6]:
datasat_path = '/content/mix_clean/'
file_list = glob.glob(datasat_path + '/*.wav')
len(file_list), file_list[0]

(3000, '/content/mix_clean/7176-88083-0001_1221-135767-0009.wav')

In [7]:
test_size = 0.30
test_files_count = int(len(file_list) *test_size)
random.shuffle(file_list)

In [8]:
test_data_files = file_list[0:test_files_count]
train_data_files = file_list[test_files_count:]
print("total no of files in test set : ", len(test_data_files))
print("total no of files in train set : ", len(train_data_files))

total no of files in test set :  900
total no of files in train set :  2100


#### LibriSpeech test-clean dataset

This is to download original librispeech dataset of test-clean set, as we already have uplaoded mixed dataset (libri2mix) above, this original dataset is required for evaluation as well

In [9]:
import torchaudio
from torchaudio.datasets import LIBRISPEECH

In [10]:
librispeech_test_clean = LIBRISPEECH(root = '/content/',
                                    url = 'test-clean',
                                    # folder_in_archive: str = 'LibriSpeech',
                                    download= True)

100%|██████████| 331M/331M [00:20<00:00, 16.8MB/s]


#### SepFormer Model Evaluation

##### Install

In [11]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [12]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 11.1 MB/s eta 0:00:00


##### imports

In [15]:
device  = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [24]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torch
from torch import tensor
from torchmetrics.audio import ScaleInvariantSignalNoiseRatio
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
# create object for si-snr & si-sdr which will be used for si-snri & si-sdri calculation
si_snr = ScaleInvariantSignalNoiseRatio().to(device)
si_sdr = ScaleInvariantSignalDistortionRatio().to(device)

##### load pretrained model

In [16]:
model = separator.from_hparams(source="speechbrain/sepformer-whamr",
                               savedir='pretrained_models/sepformer-whamr',
                          run_opts={"device":"cuda"})

hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

##### predict single sample

In [18]:
# for custom file, change path
sources_pred = model.separate_file(path='/content/mix_clean/1221-135766-0012_6930-76324-0003.wav')
# torchaudio.save("source1hat.wav", sources_pred[:, :, 0].detach().cpu(), 8000)
# torchaudio.save("source2hat.wav", sources_pred[:, :, 1].detach().cpu(), 8000)

In [19]:
mixture, sr = torchaudio.load('/content/mix_clean/1221-135766-0012_6930-76324-0003.wav')

In [20]:
sources_pred.shape, mixture.shape

(torch.Size([1, 129760, 2]), torch.Size([1, 129760]))

##### prediction and evaluation on all test set (900 files)

In [21]:
### function for SI-SNR improvement metircx using torchmetrics.audio.ScaleInvariantSignalNoiseRatio
def SI_SNRi(src_ref, src_est, mix):
  sisnr1 = si_snr(src_ref[:,:,0], src_est[:,:,0])
  sisnr2 = si_snr(src_ref[:,:,1], src_est[:,:,1])

  sisnr1b = si_snr(src_ref[:,:,0], mix)
  sisnr2b = si_snr(src_ref[:,:,1], mix)

  avg_SISNRi = ((sisnr1 - sisnr1b) + (sisnr2 - sisnr2b)) / 2
  return avg_SISNRi

# function for SI-SDR improvement metircx using torchmetrics.audio.ScaleInvariantSignalDistortionRatio
def SI_SDRi(src_ref, src_est, mix):
  sisdr1 = si_snr(src_ref[:,:,0], src_est[:,:,0])
  sisdr2 = si_snr(src_ref[:,:,1], src_est[:,:,1])

  sisdr1b = si_snr(src_ref[:,:,0], mix)
  sisdr2b = si_snr(src_ref[:,:,1], mix)

  avg_SISDRi = ((sisdr1 - sisdr1b) + (sisdr2 - sisdr2b)) / 2

  return avg_SISDRi

In [22]:
len(test_data_files), test_data_files[0]

(900, '/content/mix_clean/6930-75918-0008_4970-29093-0013.wav')

In [30]:
evaluation_results = []
for libri2mix_path in tqdm(test_data_files[:]):
  #create source 1 & source 2 flac file path
  s1_path, s2_path = libri2mix_path[:-4].split('/')[-1].split('_')
  s1_path =  '/content/LibriSpeech/test-clean/' + '/'.join(s1_path.split('-')[:2]) +'/'+ s1_path + '.flac'
  s2_path = '/content/LibriSpeech/test-clean/' + '/'.join(s2_path.split('-')[:2]) +'/'+ s2_path + '.flac'

  #read s1 & s2 flac files andresample them into 8000 sample rate as mixing is done using 8k sample rate
  s1, sample_rate1 = torchaudio.load(s1_path)
  resampler = torchaudio.transforms.Resample(sample_rate1, 8000)
  s1_8k = resampler(s1)
  s2, sample_rate2 = torchaudio.load(s2_path)
  resampler = torchaudio.transforms.Resample(sample_rate2, 8000)
  s2_8k = resampler(s2)

  #zero pad shorter source signal to match shape of both source signal
  if s1_8k.shape[-1]>s2_8k.shape[-1]:
    s2_8k = torch.nn.functional.pad(s2_8k, (0, s1_8k.shape[-1] -s2_8k.shape[-1]))
  else:
    s1_8k = torch.nn.functional.pad(s1_8k, (0, s2_8k.shape[-1] -s1_8k.shape[-1] ))

  #stack source 1 & 2 into single tensor to get source_true signal
  source_true = torch.stack((s1_8k, s2_8k), dim=2)

  #load libri2mix_path (mixed wav file)
  mixture, _ = torchaudio.load(libri2mix_path)

  #predict over mix signal using sepformer model to get source seprated signal
  sources_pred = model.separate_file(libri2mix_path)
  s1_pred = sources_pred[:, :, 0].detach().to(device)
  s2_pred = sources_pred[:, :, 1].detach().to(device)

  ## calculate si-snr & si-sdr
  si_snr1 = si_snr(s1_pred, s1_8k.to(device))
  si_snr2 = si_snr(s2_pred, s2_8k.to(device))

  si_sdr1 = si_sdr(s1_pred, s1_8k.to(device))
  si_sdr2 = si_sdr(s2_pred, s2_8k.to(device))

  #caculate si-snri & si-sdri
  si_snri = SI_SNRi(source_true.to(device), sources_pred.to(device), mixture.to(device))
  si_sdri = SI_SDRi(source_true.to(device), sources_pred.to(device), mixture.to(device))

  #append metirced to result list
  evaluation_results.append([libri2mix_path, float(si_snr1), float(si_snr2), float(si_sdr1),
                             float(si_sdr2), float(si_snri), float(si_sdri)])

100%|██████████| 900/900 [08:44<00:00,  1.72it/s]


In [31]:
evaluation_results[:2]

[['/content/mix_clean/6930-75918-0008_4970-29093-0013.wav',
  6.233344078063965,
  13.33906078338623,
  6.2329511642456055,
  13.338613510131836,
  9.858057975769043,
  9.858057975769043],
 ['/content/mix_clean/61-70968-0003_5105-28240-0000.wav',
  -30.417844772338867,
  -28.085214614868164,
  -30.42799186706543,
  -28.086088180541992,
  -29.301212310791016,
  -29.301212310791016]]

In [39]:
df = pd.DataFrame(evaluation_results, columns = ['file', 'si_snr1', 'si_snr2', 'si_sdr1', 'si_sdr2',
                                                 'si_snri', 'si_sdri'])
float_columns = df.select_dtypes(include=['float']).columns
# Apply abs function to float value columns
df[float_columns] = df[float_columns].applymap(abs)

In [40]:
df.head()

,file,si_snr1,si_snr2,si_sdr1,si_sdr2,si_snri,si_sdri
0,/content/mix_clean/6930-75918-0008_4970-29093-...,6.233344,13.339061,6.232951,13.338614,9.858058,9.858058
1,/content/mix_clean/61-70968-0003_5105-28240-00...,30.417845,28.085215,30.427992,28.086088,29.301212,29.301212
2,/content/mix_clean/1320-122617-0009_4992-23283...,5.170468,10.038000,5.167295,10.037154,7.597239,7.597239
3,/content/mix_clean/1284-1181-0015_8555-284447-...,8.473897,9.823503,8.469676,9.820310,9.011262,9.011262
4,/content/mix_clean/1580-141083-0013_237-134493...,32.142395,32.702190,32.142498,32.702633,32.403542,32.403542


In [44]:
df[float_columns].mean()

si_snr1    21.241851
si_snr2    20.721350
si_sdr1    21.234153
si_sdr2    20.705906
si_snri    20.955111
si_sdri    20.955111
dtype: float64